In [ ]:
using CairoMakie

import LilGuys as lguys
using Arya

In [ ]:
function sigma_axis(; kwargs...) 
	fig = Figure()
	ax = Axis(fig[1,1], 
		xlabel=log_r_label,
		ylabel = L"\log \Sigma\ / \textrm{(fraction/arcmin^2)}"
		;kwargs...
	)

	return fig, ax
end

In [ ]:
prof_expected = lguys.StellarProfile("/astro/dboyea/sculptor/fiducial_sample_profile.toml");

In [ ]:
log_r_label = "log r / arcmin"

In [ ]:
distance = 81

In [ ]:
function plot_finals(models, labels)
    
    fig = Figure()
	ax = Axis(fig[1,1], 
		xlabel=log_r_label,
		ylabel = L"\log \Sigma\ / \textrm{(fraction/arcmin^2)}",
		limits=((-1, 2.5), (-5, 1)),
		xgridvisible=false,
		ygridvisible=false
	)

	errscatter!(prof_expected.log_r, prof_expected.log_Sigma,
		yerr=prof_expected.log_Sigma_err,
		color=:black,
        label="J+24",
	)

    for i in eachindex(models)
    	lines!(models[i].log_r, models[i].log_Sigma, 
    			label=labels[i])
    end

	axislegend()

	fig
end

In [ ]:
function plot_ini_fin(prof_i, prof_f, model)
    
    fig = Figure()
	ax = Axis(fig[1,1], 
		xlabel=log_r_label,
		ylabel = L"\log \Sigma\ / \textrm{(fraction/arcmin^2)}",
		limits=((-1, 2.5), (-5, 1)),
		xgridvisible=false,
		ygridvisible=false
	)

	errscatter!(prof_expected.log_r, prof_expected.log_Sigma,
		yerr=prof_expected.log_Sigma_err,
		color=:black,
        label="J+24",
	)

    lines!(prof_i.log_r, prof_i.log_Sigma, 
            label="initial")

    lines!(prof_f.log_r, prof_f.log_Sigma, 
            label="final")

    x = LinRange(-2, 2.5, 1000)
    r = 10 .^ x
    y = lguys.calc_Σ.(model, r)

    r_scale = lguys.kpc_to_arcmin(1, distance)

    x = x .+ log10(r_scale)
    y ./= y[1]

    lines!(x, log10.(y), linestyle=:dot, color=:black, label="expected")
	axislegend()

	fig
end

In [ ]:
using Measurements

In [ ]:
function get_log_Σ_0(prof, χ2_max=1)
    log_r = prof.log_r

    @assert issorted(log_r)
    local log_Σ_0, log_Σ_0_err
    
    for i in eachindex(log_r)
        filt = 1:i
        log_Σ_0 = lguys.mean(prof.log_Sigma[filt] .± prof.log_Sigma_err[filt])
        log_Σ_0_err = Measurements.uncertainty.(log_Σ_0)
        log_Σ_0 = Measurements.value(log_Σ_0)
        χ2 = @. (log_Σ_0 - prof.log_Sigma)^2 / (prof.log_Sigma_err + log_Σ_0_err)^2

        χ2_norm = lguys.mean(χ2[filt])

        if χ2_norm .> χ2_max
            @info "max radius of $(log_r[i])"
            break
        end
    end
    
    return log_Σ_0, log_Σ_0_err
end

In [ ]:
get_log_Σ_0(profiles[1])

In [ ]:
function get_normalization(prof_f, prof_expected=prof_expected)
    log_Σ_0_obs, _ = get_log_Σ_0(prof_expected)
    log_Σ_0, _ = get_log_Σ_0(prof_f)

    return log_Σ_0_obs - log_Σ_0
end

## V=32, r=5.9 (the initial model)

In [ ]:
model_dir = "/astro/dboyea/sculptor/orbits/orbit1/1e6_V32_r5.4/stars/"
stars_dir = "/astro/dboyea/sculptor/isolation/1e6/fiducial/stars_ana/"

In [ ]:
models = [
    "exp2d_rs0.07",
    "exp2d_rs0.10",
    "exp2d_rs0.20",
]

In [ ]:
names_i = [model_dir] .* models .* ["_i_profile.toml"]
names_f = [model_dir] .* models .* ["_profile.toml"]
names_profiles = [stars_dir] .* models .* [".toml"]

In [ ]:
profiles = lguys.StellarProfile.(names_f);
profiles_i = lguys.StellarProfile.(names_i);

In [ ]:
profile_an = lguys.StellarProfile("/astro/dboyea/dwarfs/notebooks/anake_test_profile.toml");

In [ ]:
expected = lguys.load_profile.( names_profiles)

In [ ]:
plot_finals([profiles; profile_an], [models; "ananke"])

In [ ]:
plot_finals(profiles_i, models)

In [ ]:
i = 1
plot_ini_fin(profiles_i[i], profiles[i], expected[i])

### King profiles

In [ ]:
models = [
    "king_rs0.10",
    "king_rs0.15",
]

In [ ]:
names_i = [model_dir] .* models .* ["_1_profile.toml"]
names_f = [model_dir] .* models .* ["_profile.toml"]
names_profiles = [stars_dir] .* models .* [".toml"]

In [ ]:
profiles = lguys.StellarProfile.(names_f);
profiles_i = lguys.StellarProfile.(names_i);

In [ ]:
expected = lguys.load_profile.( names_profiles)

In [ ]:
plot_finals(profiles, models)

In [ ]:
plot_finals(profiles_i, models)

In [ ]:
expected[2].R_t

In [ ]:
lguys.kpc_to_arcmin(0.75, 81)

In [ ]:
log10(31)

# Orbit studies

## V=32, r=2.4

In [ ]:
models = [
    "exp2d_rs0.10",
]

In [ ]:
model_dir = "/astro/dboyea/sculptor/orbits/orbit1/1e6_V31_r3.2/stars/"
stars_dir = "/astro/dboyea/sculptor/isolation/1e6/V31_r3.2/stars/"

names_i = [model_dir] .* models .* ["_i_profile.toml"]
names_f = [model_dir] .* models .* ["_profile.toml"]
names_profiles = [stars_dir] .* models .* [".toml"]

In [ ]:
expected = lguys.load_profile.( names_profiles)

In [ ]:
profiles = lguys.StellarProfile.(names_f);

In [ ]:
profiles_i = lguys.StellarProfile.(names_i);

In [ ]:
norms = get_normalization.(profiles)

In [ ]:
[prof.log_Sigma .+= norm for (prof, norm) in zip(profiles, norms)]

In [ ]:
[prof.log_Sigma .+= norm for (prof, norm) in zip(profiles_i, norms)]

In [ ]:
prof_fit = lguys.KingProfile(M=1, R_s=11, R_t=25)

In [ ]:
r_b = 76

In [ ]:
i = 1
plot_ini_fin(profiles_i[i], profiles[i], expected[i])